In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import cv2
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, precision_recall_fscore_support, roc_curve, auc, precision_score,recall_score, f1_score
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

In [ ]:
cat_file_path = "/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat"
dog_file_path = "/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog"

In [ ]:
import plotly.express as px

class_names = ['Cat', 'Dog'] 
n_dogs = len(os.listdir(cat_file_path))
n_cats = len(os.listdir(dog_file_path))
n_images = [n_cats, n_dogs]

fig = px.bar(x=class_names, y=n_images, color=class_names, labels={'x': 'Class', 'y': 'Number of Images'}, title='Dog vs Cat')
fig.show()

In [ ]:
data = []

In [ ]:
print(data)

In [ ]:
for img in os.listdir(cat_file_path):
        img_path = os.path.join(cat_file_path, img)
        label = 0
        arr = cv2.imread(img_path)
        if arr is None:
            print(f"Error loading image: {img_path}")
        else:
            augmented_image = random_rotation(arr)
            augmented_image = horizontal_flip(augmented_image)
            augmented_image = vertical_flip(augmented_image)
            augmented_image = cv2.resize(augmented_image, (150, 150))
            new_arr = cv2.resize(arr, (150, 150))
            data.append([new_arr, label])
            data.append([augmented_image, label])

In [ ]:
            
for img in os.listdir(dog_file_path):
        img_path = os.path.join(dog_file_path, img)
        label = 1
        arr = cv2.imread(img_path)
        if arr is None:
            print(f"Error loading image: {img_path}")
        else:
            # Apply transformations
            augmented_image = random_rotation(arr)
            augmented_image = horizontal_flip(augmented_image)
            augmented_image = vertical_flip(augmented_image)
            augmented_image = cv2.resize(augmented_image, (150, 150))
            new_arr = cv2.resize(arr, (150, 150))
            data.append([new_arr, label])
            data.append([augmented_image, label])

In [ ]:
print(data[0])

In [ ]:
len(data)

In [ ]:
random.shuffle(data)

In [ ]:
data[0]

In [ ]:
train_data,test_data  = train_test_split(data, train_size=0.8, random_state=42)

In [ ]:
len(train_data)

In [ ]:
train_data[0]

In [ ]:
len(test_data)

In [ ]:
test_data[0]

In [ ]:
X_train = []
y_train = []

for features,labels in train_data:
    X_train.append(features)
    y_train.append(labels)

In [ ]:
X_train[0]

In [ ]:
y_train[0]

In [ ]:
X_test = []
y_test = []

for features,labels in test_data:
    X_test.append(features)
    y_test.append(labels)

In [ ]:
X_test[0]

In [ ]:
y_test[0]

In [ ]:
X_test = np.array(X_test)
y_test = np.array(y_test)
X_train = np.array(X_train)
y_test = np.array(y_test)

In [ ]:
del test_data
del train_data

In [ ]:
del data

In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
len(X_train)

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.reshape((len(X_train), 150*150*3))
X_test = X_test.reshape((len(X_test), 150*150*3))

In [ ]:
X_train.shape

In [ ]:
X_train = X_train/255
X_test = X_test/255

In [ ]:
X_train[0]

In [ ]:
def convert_to_categorical(labels, num_classes=2):
    one_hot_labels = np.zeros((len(labels), num_classes))

    for i, label in enumerate(labels):
        one_hot_labels[i, label] = 1

    return one_hot_labels

In [ ]:
y_train = convert_to_categorical(y_train, 2)
y_test = convert_to_categorical(y_test, 2)

In [ ]:
y_train[1]

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
class ANN:
  def __init__(self):

    self.learning_rate = 0.01
    self.epochs = 50

    # Neural network architecture parameters
    self.input_size = 150*150*3  # 150*150
    self.hidden_size = 5
    self.output_size = 5  

    #Initialize weights and biases
    np.random.seed(42)
    self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size)
    self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size)
    self.bias_hidden = np.zeros((1, self.hidden_size))
    self.bias_output = np.zeros((1, self.output_size))

  def sigmoid(self,x):
    return 1 / (1 + np.exp(-x))

  def sigmoid_derivative(self,x):
      return x * (1 - x)

  def binary_cross_entropy(self,y_true, y_pred):
    # Avoid division by zero
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss

  def train(self,x_train,y_train):
      # Training the network
      X_train = x_train
      y_train = y_train
      for e in range(self.epochs):
          total_loss = 0
          for i in range(len(X_train)):
              # Forward pass
              input_layer = np.array([X_train[i]])
              hidden_layer = self.sigmoid(np.dot(input_layer, self.weights_input_hidden) + self.bias_hidden)
              output_layer = self.sigmoid(np.dot(hidden_layer, self.weights_hidden_output) + self.bias_output)
              # Backpropagation
              loss = self.binary_cross_entropy(y_test,output_layer)
              total_loss += loss
              output_error = y_train[i] - output_layer
              output_delta = output_error * self.sigmoid_derivative(output_layer)

              hidden_error = output_delta.dot(self.weights_hidden_output.T)
              hidden_delta = hidden_error * self.sigmoid_derivative(hidden_layer)

              # Update weights and biases
              self.weights_hidden_output += hidden_layer.T.dot(output_delta) * self.learning_rate
              self.bias_output += np.sum(output_delta, axis=0, keepdims=True) * self.learning_rate
              self.weights_input_hidden += input_layer.T.dot(hidden_delta) * self.learning_rate
              self.bias_hidden += np.sum(hidden_delta, axis=0, keepdims=True) * self.learning_rate
            
          # Output epoch results
          total_loss /= len(X_train)
          print(f"Epoch {e+1}/{self.epochs} loss - {total_loss}")

  def evaluate(self,X_test,y_test):
      # Evaluating the model
      correct_predictions = 0
      pred = []
      for i in range(len(X_test)):
          input_layer = np.array([X_test[i]])
          hidden_layer = self.sigmoid(np.dot(input_layer, self.weights_input_hidden) + self.bias_hidden)
          output_layer = self.sigmoid(np.dot(hidden_layer, self.weights_hidden_output) + self.bias_output)
          pred.append(output_layer)
          if np.argmax(output_layer) == np.argmax(y_test[i]):
              correct_predictions += 1

      print(f"Test accuracy: {correct_predictions / len(y_test)}")
      return pred

In [ ]:
ann_model = ANN()

In [ ]:
ann_model.train(X_train,y_train)

In [ ]:
pred = ann_model.evaluate(X_test,y_test)

In [ ]:
pred = np.array(pred).squeeze()

In [ ]:
top1_accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(pred, axis=1))
print(f"Accuracy: {top1_accuracy * 100:.2f}%")

In [ ]:
precision, recall, f1_score, _ = precision_recall_fscore_support(np.argmax(y_test, axis=1), np.argmax(pred, axis=1), average='weighted')
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")

In [ ]:
conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(pred, axis=1))
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
num_classes = 2
TP = np.zeros(num_classes)
TN = np.zeros(num_classes)
FP = np.zeros(num_classes)
FN = np.zeros(num_classes)

for cls in range(num_classes):
    TP[cls] = conf_matrix[cls, cls]
    TN[cls] = np.sum(conf_matrix) - np.sum(conf_matrix[cls, :]) - np.sum(conf_matrix[:, cls]) + conf_matrix[cls, cls]
    FP[cls] = np.sum(conf_matrix[:, cls]) - conf_matrix[cls, cls]
    FN[cls] = np.sum(conf_matrix[cls, :]) - conf_matrix[cls, cls]

for cls in range(num_classes):
    print(f"Class {cls}:")
    print(f"  True Positives (TP): {TP[cls]}")
    print(f"  True Negatives (TN): {TN[cls]}")
    print(f"  False Positives (FP): {FP[cls]}")
    print(f"  False Negatives (FN): {FN[cls]}\n")


In [ ]:
n_classes = y_test.shape[1]

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves
plt.figure(figsize=(10, 8))
lw = 2
for i, color in zip(range(n_classes),['aqua', 'darkorange']):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], '--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC')
plt.legend(loc="lower right")
plt.show()